# CrossSection Post-processing

## Basics

### Revision control

Version 6.4
- Combine different contact bodies
- NOT DONE: Fix hardcoding of stress
- NOT DONE: Fix so that file written can take different number of columns for each load case - put into dataframe and use .to_csv?
- Read positions based on curvature gradient?

Version 6.3
- Added stress from mentat post-processing in dataframe (layer and position is hardcoded to Cauchy 11 Pref TA Inner max EF

Version 6.2
- General update of code
- Added possibility to plot for each part in each case

Version 6.1
- Worked throught the code with updates
- Improved readability and robustness

Version 6.0
- Based on version 5.2
- Updated with better documentation as being used
- Included and refreshed parts used for axisymmetric post-processing

### Import

In [1]:
# Modules
import pathlib
import plotly
import logging
import re
import os

# Modules with short names
import pandas as pd
import plotly.graph_objs as go
import ipywidgets.widgets as widgets

# Packes from modules
from tqdm.notebook import tqdm
from ipywidgets import Layout
from IPython.display import display, clear_output, HTML
plotly.offline.init_notebook_mode(connected=True)

# Initiate logging
logging.basicConfig(filename=r'CS_posting_v2.log',
                    level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')

logging.info('STARTED')

### Functions

In [2]:
# file_dir = pathlib.Path(r'C:\Projects\1638\Marc\Model_EF\SNA_2p_ef_t8_gap2_job1_10559cb.csv')
combine = ['Flextensile1', 'Flextensile2']

"""
Reads .csv-file produced by CrossSection
Renames columns to more readable names
Calculates "total force"

Input:
- file_dir : directory of file (pathLib)
- combine : combine all bodies matching the names in combine (list)

TODO: Return von mieses
"""

# Definitions
name, new_name = '', ''

# Specify whether to look after element set or contact body name
if os.path.isfile(file_dir):
    if 'sum' in file_dir.name:
        cs_type ='Element Set'
    elif 'cb' in file_dir.name:
        cs_type = 'Contact Body'
    else:
        raise Exception(f'Check {file_dir} name - should contain cb or sum')
else:
    raise Exception(f'cs_reader did not find {file_dir}')

# Read csv file
df = pd.read_csv(file_dir)
df.dropna(how='all', axis=1, inplace=True) # removes columns with nan only

# Combine columns
df_combined = pd.DataFrame()
if isinstance(combine, list):
    for layer in combine:
        # Find columns to combine
        df_comb = df[[col for col in df.columns if layer in col]]

        # Find properties
        props = set(col.split('.')[0] for col in df_comb.columns)

        # Iterate over properties and combine columns with match
        for prop in props:
            comb_match = f'{prop}.{layer}'
            df_combined[comb_match] = df_comb[[col for col in df_comb.columns if comb_match in col]].sum(axis=1)

    # Find columns that have been combined and that are excessive
    columns_final = df_combined.columns
    columns_all = [col for col in df.columns if any([layer in col for layer in combine])]
    columns_excessive = list([l for l in columns_all if l not in columns_final])

    # Remove excessive columns and change content in matched columns
    df.drop(columns_excessive, inplace=True, axis=1)
    
    # Changes columns
    for col in columns_final:
        df[col] = df_combined[col]

# Find element data: number, names and order
for c in df.columns.values:
    if cs_type in c:
        name = df[c][1].strip()
    if name:
        col_name = c.split('.')[0]
        new_name = '.'.join([col_name, name])
        df.rename(columns={c : new_name}, inplace=True)


# Calc additional parameters for each part
cbodies = set([n.split('.')[-1] for n in df.columns.values if '.' in n])

for p in sorted(cbodies):
    # Total force
    df['.'.join(['Ftot', p])] = (df['.'.join(['Fx', p])]**2 + 
                                 df['.'.join(['Fy', p])]**2 + 
                                 df['.'.join(['Fz', p])]**2).pow(1/2)

    # Calc stiffness for each part
    df['.'.join(['EI', p])] = (df['.'.join(['My', p])] / df['.'.join(['K1_2'])]).replace([np.inf], 0.0)

NameError: name 'file_dir' is not defined

In [3]:
def cs_reader(file_dir, combine=''):
    """
    Reads .csv-file produced by CrossSection
    Renames columns to more readable names
    Calculates "total force"

    Input:
    - file_dir : directory of file (pathLib)
    - combine : combine all bodies matching the names in combine (list)

    TODO: Return von mieses
    """

    # Definitions
    name, new_name = '', ''

    # Specify whether to look after element set or contact body name
    if os.path.isfile(file_dir):
        if 'sum' in file_dir.name:
            cs_type ='Element Set'
        elif 'cb' in file_dir.name:
            cs_type = 'Contact Body'
        else:
            raise Exception(f'Check {file_dir} name - should contain cb or sum')
    else:
        raise Exception(f'cs_reader did not find {file_dir}')

    # Read csv file
    df = pd.read_csv(file_dir)
    df.dropna(how='all', axis=1, inplace=True) # removes columns with nan only

    # Combine columns
    df_combined = pd.DataFrame()
    if isinstance(combine, list):
        for layer in combine:
            # Find columns to combine
            df_comb = df[[col for col in df.columns if layer in col]]

            # Find properties
            props = set(col.split('.')[0] for col in df_comb.columns)

            # Iterate over properties and combine columns with match
            for prop in props:
                comb_match = f'{prop}.{layer}'
                df_combined[comb_match] = df_comb[[col for col in df_comb.columns if comb_match in col]].sum(axis=1)

        # Find columns that have been combined and that are excessive
        columns_final = df_combined.columns
        columns_all = [col for col in df.columns if any([layer in col for layer in combine])]
        columns_excessive = list([l for l in columns_all if l not in columns_final])

        # Remove excessive columns and change content in matched columns
        df.drop(columns_excessive, inplace=True, axis=1)
        
        # Changes columns
        for col in columns_final:
            df[col] = df_combined[col]


    # Find element data: number, names and order
    for c in df.columns.values:
        if cs_type in c:
            name = df[c][1].strip()
        if name:
            col_name = c.split('.')[0]
            new_name = '.'.join([col_name, name])
            df.rename(columns={c : new_name}, inplace=True)


    # Calc additional parameters for each part
    cbodies = set([n.split('.')[-1] for n in df.columns.values if '.' in n])

    for p in sorted(cbodies):
        # Total force
        df['.'.join(['Ftot', p])] = (df['.'.join(['Fx', p])]**2 + 
                                     df['.'.join(['Fy', p])]**2 + 
                                     df['.'.join(['Fz', p])]**2).pow(1/2)

        # Calc stiffness for each part
        df['.'.join(['EI', p])] = (df['.'.join(['My', p])] / df['.'.join(['K1_2'])]).replace([np.inf], 0.0)
        
    return df, cbodies


def find_files(run_dir, *args):
    '''
    Function that finds all files with specified ending
    
    :param run_dir: Directory containing files
    :param args: Arguments which run_dir must contain
    
    :return files: Files containing arguments

    '''
    files = []
    
    for arg in args:
        file_paths = sorted(run_dir.rglob(f'*{arg}*'))
        files += [p.relative_to(run_dir) for p in file_paths]
    
    return files


def find_ranges(my_dict, x_divisor=500, y_divisor=50000):
    '''
    Function that finds the range of the x- and y-axis based on data in my_dict
    
    :dict my_dict: Dictionary to process
    :param x_divisor: Number the x-range are rounded up to
    :param y_divisor: Number the y-range are rounded up to
    
    :return x_range, y_range: Two lists with min/max
    '''
    
    x_max = max(list(my_dict.values())[0].columns.values)
    x_range = [0, ((x_max // x_divisor + 1) * x_divisor)]

    y_max = 0
    y_min = 0
    for layer in my_dict.keys():
        if 'SUM' not in layer:
            for col in my_dict[layer].columns.values:
                my_list = list(my_dict[layer][col])
                my_min = min(my_list)
                my_max = max(my_list)

                if my_min < y_min:
                    y_min = my_min

                if my_max > y_max:
                    y_max = my_max

    y_range_max = ((y_max // y_divisor + 1) * y_divisor)
    y_range_min = ((abs(y_min) // y_divisor + 1) * y_divisor) * -1

    y_range = [y_range_min, y_range_max]
    
    return x_range, y_range


def read_results(case_path, res):
    '''
    Function that reads all .csv-files in case_path
    
    :param case_pth: path with .csv-files
    :param res: result to be extracted from csv-file
    
    TODO: 
    - Hva hvis det er .csv-filer som ikke inneholder resultater i mappen?
    '''
    
    # Initialize vectors
    res_dict = {}
    temp_dict = {}
    time = pd.DataFrame()
    displacements = pd.DataFrame()
    temp_df = pd.DataFrame()
    
    # Find .csv-files with results
    csv_files = find_files(case_path, 'csv')
    
    # Find names of contact bodies
    df = pd.read_csv(case_path / csv_files[0]) # NEW
    names = [df[cb][0].lstrip() for cb in df.columns.values if 'Contact Body' in cb]
    
    # Initialize dictionary with dataframe for each layer
    for name in names:
        res_dict[name] = pd.DataFrame()
    
    # Extract desired result from each file
    for csv_file in csv_files:
        csv_name = csv_file.stem
        node = csv_name.split('_')[-1].replace('cb', '')
        
        csv_path = case_path / csv_file
        df = pd.read_csv(csv_path)
        
        temp_dict[node] = extract_results(df, res)
    
    # Re-format data
    for node in temp_dict.keys():
        displacements[node] = temp_dict[node]['X-disp']
        
        for name in names:
            res_dict[name][displacements[node][0]] = temp_dict[node][name]
        
    res_dict['X-disp'] = displacements
    
    # Sort columns (with forces for axial positions along pipe) numerically for plotting
    for name in res_dict.keys():
        res_dict[name] = res_dict[name].sort_index(axis=1)
    
    return res_dict


def extract_results(df, res):
    '''
    Function that extract result result from dataframe
    
    :param df: DataFrame wtih all results
    :param res: Result to be extracted
    
    :return df_res: DataFrame with selected results
    '''
    
    if not isinstance(res, str):
        sys.exit('{} is not a string'.format(res))
        
    df_res = pd.DataFrame()
    node_disp = pd.DataFrame()
    
    df_res['Time'] = df['time']
    df_res['X-disp'] = df['Origo X GCS']
    
    names = [df[cb][0].lstrip() for cb in df.columns.values if 'Contact Body' in cb] # extract names of contact bodies
    res_cols = [col for col in df.columns if res in col]

    if len(res_cols) == len(names):
        for i, col in enumerate(res_cols):
            df_res[names[i]] = df[col]

        return df_res
    else:
        sys.exit('Can not extract {} from dataframe'.format(res))
        
        
def trim_list(my_list, rem_strings):
    '''
    Function that removes strings from a list
    
    :list my_list: List with strings
    :list rem_strings: List with strings to be removed.
    
    :return my_list: List with strings removed
    '''
    
    for s in rem_strings:
        if s in my_list:
            my_list.remove(s)
    return my_list


def strain2stress(df):
    '''
    Function used to show stress of material instead of strain,
    which is given by CS as default
    
    df : 
        
    '''
    
    df_res = {}
    # read excel
    mat_316 = pd.ExcelFile(os.path.join(project_folder, '1324_Material.xlsx')).parse('316')
    
    for case in df.keys():
        df_stress = pd.DataFrame()
        for col in df[case].columns.values:
            df_stress[c] = np.interp(df[case][col], 
                                     mat_316['True plastic strain'], 
                                     mat_316['True stress'])

        df_res[case] = df_stress.where(df_stress > mat_316['True stress'][0])
        
    return df_res


def plot_data(x, y, res, keep='', remove='', plot_title=''):
    """
    Function used to hide default input from main cell
    
    x : dataframe to be plotted along x-axis
    y : dataframe to be plotted along y-axis
    res : result to be plotted (e.g. 'Fx')
    keep : if to keep only a selected number of sets/cbs (e.g. 'brick1', 'brick2')
    remove : if to remove selected sets/cbs (e.g. 'none', 'liner')
    plot_title : title of plot. default is case name
    """
    
    if not isinstance(res, str):
        raise Exception('Error in plot_data - res not string')
    
    df_plot = df_filter(y, res, keep, remove)
    
    data = []
    for i, plot_list in enumerate(df_plot.columns.values):
        name = plot_list.split('.')[-1]
        if name[0:-1] in set_explanation.keys():
            name = set_explanation[name[0:-1]] + name[-1]
        else:
            name = name.capitalize()
            
        trace = go.Scatter(mode=line_styles[0],
                           x = x,
                           y = df_plot[plot_list],
                           name = name,
                           line = {'color' : colors_4s[i]}
                          )
        data.append(trace)
                  
    try:
        yaxis_title = fm_units[plot_list.split('.')[0]]
    except (KeyError, UnboundLocalError):
        yaxis_title = 'Plastic strain [-]'
        #yaxis_title = keep
        
    layout = go.Layout(title=plot_title,
                       xaxis=dict(title='Contact force [N]'),
                       #yaxis=dict(title='Plastic strain [-]'))
                       yaxis=dict(title=yaxis_title))
    
    plotly.offline.iplot(go.Figure(data=data, layout=layout))

    
def read_excel_results(excel_file_dir, res, cases):
    '''
    Reads each sheet in excel_file_dir, and only columns matching 'res' will be extracted.
    Sheet name will be changed to readable format based on dictionary 'cases'
    
    TODO:
    - What if excel sheet gives multiple matches? E.g. if both Contact Force X & Y is present?
    '''
    df_excel = {}
    xl = pd.ExcelFile(excel_file_dir)
        
    # For each sheet - this gives result on format df_excel[case][result]
    for sheet in xl.sheet_names:
        if any([sheet in case for case in cases.values()]):
            df = xl.parse(sheet)
            df.dropna(how='all', axis=1, inplace=True)

            df_excel[sheet] = df
        
    return df_excel


def df_filter(df, res='', keep='', remove=''):
    """
    For the dataframe 'df', ONLY the result 'res' will be kept, and columns containing words in 'remove' will be removed
    
    df : dataframe to filter
    res : result to be kept
    keep : strings to be kept
    remove : strings to be removed
    
    TODO: df.drop lines can def be improved wrt readability
    """
    logging.info(f'{keep}')
    # Check type - converts to lists    
    if isinstance(res, str):
        res = [res]
    
    if isinstance(remove, str):
        remove = [remove]
        
    if isinstance(keep, str):
        keep = [keep]

    # Operations
    for r in res:
        if r != '':
            df = df.filter(regex=r)
            
    if keep != ['']:
        # Default keep
        keep = ['inc', 'time', 'K1_2', 'Ovality Y'] + keep
        
        # Keep
        logging.info(f'{list(df.columns.values)}')
        df = df.drop([i for i in df.columns.values if not any(y for y in keep if i.lower().endswith(y.lower()))], axis=1)
        logging.info(f'{keep}')
    if remove != ['']:
        # Remove
        df = df.drop([i for i in df.columns.values if any(y for y in remove if i.lower().endswith(y.lower()))], axis=1)
    
    return df


def df_combine_csv(case, excel_file, post_folder):
    """
    
    ! U N F I N I S H E D   A N D   U N U S E D !
    
    For cases where multiple independent cuts and sets have been run, 
    this function collects and stores all sets in a dataframe
    
    case : 
    excel_file : 
    post_folder : 
    """
    
    df_set = pd.DataFrame()
    xl = pd.ExcelFile(os.path.join(project_folder, excel_file))
    
    # extract info from excel file
    for sheet in xl.sheet_names:
        case_folder = os.path.join(sheet, post_folder)
        df_excel = xl.parse(sheet)
        
        if case in sheet:
            # read each csv-file and combine into one df
            for elset, csv in zip(df_excel['set'], df_excel['csv-file']):
                df = df_reader(case_folder, csv)
                df_set = pd.concat([df_set, df.filter(regex=elset)], axis=1, sort=False)
        else:
            print('{} not in {}'.format(case, sheet))
  
    return df_set

## Project Structure

### Definitions

In [4]:
# Standard defintions
# TODO : can be written to {'Fx' : {'Force X' : 'Force [N]'}} and refer to ass fm['Fx'].key 
fm = {'EI' : 'Stiffness', 'Fx' : 'Axial Force X', 'Fy' : 'Transverse Force Y', 'Fz' : 'Shear Force Z', 'Ftot' : 'Total force',
      'Mx' : 'Torque', 'My' : 'Moment Y', 'Mz' : 'Moment Z', 'K1_2' : 'Curvature', 'K3_4' : 'Curvature', 'Ovality Y' : 'Ovality', 'Iyy' : 'Iyy', 'Iyz' : 'Iyz', 'Izz' : 'Izz',
      'Av Pl Strain' : 'Average plastic strain', 'Stress' : 'Stress',
      'Max Pl Strain' : 'Max plastic strain'}

fm_units = {'EI' : 'Stiffness [Nmm2]',  'Fx' : 'Force [N]', 'Fy' : 'Force [N]', 'Fz' : 'Force [N]', 'Ftot' : 'Total Force [N]',
            'Mx' : 'Moment [Nmm]', 'My' : 'Moment [Nmm]', 'Mz' : 'Moment [Nmm]', 'K1_2' : 'Curvature [1/m]', 'K3_4' : 'Curvature [1/m]', 'Ovality Y' : 'Ovality [-]',
            'Av Pl Strain' : 'Plastic strain [-]', 'Stress' : 'Stress (MPa)', 'Iyy' : 'Iyy', 'Iyz' : 'Iyz', 'Izz' : 'Izz',
            'Max Pl Strain' : 'Plastic strain [-]'}

# Names used in plot instead of names used in model, e.g. {'model_name' : 'Plot name'}
set_explanation = {'fbr' : 'Front brick tooth',
                   'flh' : 'Front liner holder'
                  }

colors_4s = ['#00a0b0', '#f3776f', '#012b5d', '#009ddc', '#00a88f', '#faa634', '#c0c1ba', '#f8ed9b',
             '#00a0b0', '#f3776f', '#012b5d', '#009ddc', '#00a88f', '#faa634', '#c0c1ba', '#f8ed9b']

line_styles = ['lines', 'lines+markers', 'markers', 'lines', 'lines+markers', 'markers', 'lines', 'lines+markers', 'markers']

### Input

In [5]:
# Input
project = r'D:\Projects\1978_BP_Tortue_EF_Torque\Phase1'
runs = ['Analysis'] # Folder with runs

# Case names ('marc_name_job1' : 'Explaining text in plot')
cases = {'full_t6_job1' : '3D both layers<br>Negative direction',
         'full_t8_oppo_job1' : '3D both layers<br>Positive direction'
        }


# Optional - cases to be excluded from post-processing
exclude_cases = []

# Optional - (here : local cases) to refer to in plots
cases_local = {'Sag_3_pitch' : 'gk_sag_t6_3p_local_t3'}


# Generate paths
project_folder = pathlib.Path(r'C:\Projects', project)

## 3D

### Read data from excel file

In [161]:
if True == False:
    # Excel file with results to be read
    excel_file = '1639_global_results.xlsx'
    extract_results = ['Time', 'Contact Force', 'Stress', 'Bending Moment']
    excel_file_dir = project_folder / excel_file

    # Extract results
    try:
        df_results = read_excel_results(excel_file_dir, extract_results, cases)
    except Exception:
        pass

    # OPTIONAL - extract local results
    try:
        if isinstance(cases_local, dict):
            df_local = read_excel_results(excel_file_dir, extract_results, cases_local)
    except Exception as e:
        logging.warning(f'{e}')
        print(f'{e}')
        pass

    # # Plot each case in df_results['Contact Force']
    # data = []
    # for i, case in enumerate(df_results.keys()):
    #     trace = go.Scatter(mode=line_styles[0],
    #                        x = df_results[case]['Time'],
    #                        y = df_results[case]['Contact Force'],
    #                        name = case,
    #                        line = {'color' : colors_4s[i]}
    #                       )
    #     data.append(trace)

    # # Define layout of plot
    # layout = go.Layout(title='Contact force comparison',
    #                    xaxis=dict(title='Time [s]', range=[1.2, 1.8]),
    #                    yaxis=dict(title='Contact force [N]', range=[0, 300000]))

    # # plotly.offline.iplot(go.Figure(data=data, layout=layout))

### Read data from csv-files

Intention of following cell: Read all CrossSection files in specified directory and sort them to correct model. All .csv- files which has a corresponding cb.res files are taken as cs -files

In [28]:
# Initialization
sorted_csv, df_case, df_stress, increments, cbodies = {}, {}, {}, {}, {}
a=0
# Find csv files
csv_files, history_list = [], []
for run in runs:
    
    run_folder = project_folder / run
    
    # Find csv files
    all_csv_files = [run_folder / f.lower() for f in os.listdir(run_folder) if len(re.findall(r'\d+cb.csv', f)) == 1]
    csv_files += [f for f in all_csv_files if os.path.isfile(run_folder / str(f).replace('cb.csv', 'cb.res'))] # correct
#     csv_files += [f for f in all_csv_files if os.path.isfile(run_folder / str(f).replace('cb.csv', 'cb.csv'))]
    
    # Find histroy files
    history_list += run_folder.glob('History-plot*')
    
history_files = {hf.stem.lower() : hf for hf in history_list}

# Remove cases to be exluded
csv_files = [csv for csv in csv_files if not any([ex.lower() in csv.stem.lower() for ex in exclude_cases])]

# Sort csv so that csv-files are tied to correct model
for csv_file in csv_files:
    csv_info = csv_file.stem.rpartition('_')
    model, csv = csv_file.parent / csv_info[0], csv_info[2]
    
    if model not in sorted_csv.keys():
        sorted_csv[model] = [csv]
    else:
        sorted_csv[model].append(csv)

# Get dataframe for each case
for case in tqdm(sorted_csv.keys(), leave=False):
    
#     if not any([ntag in str(case) for ntag in ['t13', 't14', 't15']]):
#         continue
        
    # For each case file / CS cut
    df_csv = {}
    for csv_file in tqdm(sorted_csv[case], leave=False):

        # Find directory to csv and read csv file
        csv_file_dir = case.parent / ''.join([case.stem, '_', csv_file, '.csv'])
        
        try:
            df_temp, cbodies[case.stem.lower()] = cs_reader(csv_file_dir)
            # combine=['Epoxy_inner1', 'Epoxy_inner2', 'Epoxy_outer1', 'Epoxy_outer2', 'Epoxy_outer3']
        except Exception as e:
            logging.warning(f'{e} : {csv_file_dir} is empty')
            continue

        # If global case is also run locally, and local run results exists: Add stress
        try:
            case_run_local = cases_local[case]
            if all([case in cases_local.keys(), case_run_local in df_local.keys()]):
                # Add stress
                df_temp['Stress.Pressure_armour'] = np.interp(df_results[case_run]['Time'],
                                                              df_local[case_run_local]['Time'].dropna(),
                                                              df_local[case_run_local]['Stress'].dropna())
        except (KeyError, NameError):
            pass

        # One df for each position
        pos = df_temp['Origo X GCS'][0]
        df_csv[f'{pos:.0f}'] = df_temp
        df_case[case.stem] = df_csv
        
        # Extract increments
        if case.stem not in increments:
            increments[case.stem] = df_temp['inc'].iloc[-1]
            
        
    # Add stress
    hf = f'history-plot_' + str(case.stem)
    if hf in history_files.keys():
        history_file = history_files[hf]
        
        # Read crossection data and use same time step
        dfs = pd.read_csv(history_file)
        dfs = dfs.loc[dfs['Time'].round(3).isin(df_temp['time'].round(3))]

        # For now, only takes Comp 11 cauchy and Tensile inner (max - EF)
        dfs = dfs[['Time'] + [col for col in dfs.columns.values if 'Comp 11 of Cauchy Stress in Preferred Sys' in col]]
        if any(['Tensile inner (max - EF)' in col for col in dfs.columns.values]):
            df_stress[case.stem.lower()] = pd.DataFrame()
            df_stress[case.stem.lower()]['time'] = dfs['Time']
            df_stress[case.stem.lower()][f'Stress.Flextensile1'] = dfs[[col for col in dfs.columns if 'Tensile inner (max - EF)' in col]].max(axis=1)
            df_stress[case.stem.lower()].reset_index(inplace=True)

#     if 't12' in str(case.stem):
#         break
# Resulting format is df_case[case][position][result]


# INDEX ON DF_STRESS IS WRONG - WHY?

#### Init

In [29]:
# Plot info
max_increments = max([inc for inc in increments.values()])

x_axis_options = ['Time', 'Curvature', 'Length']

# Positions
positions_all, cbodies_all = [], []
for case in df_case.keys():
    # Get positions for this case
    positions_all += [f'{int(p)}' for p in df_case[case].keys()]
    
    # Get cbodies
    try:
        cbodies_all += cbodies[case]
    except KeyError:
        pass

positions_all = list(set(positions_all))
positions_all.sort(key = int)

cbodies_all = list(set(cbodies_all))

# W I D G E T S

# Cases
rows_widgets = max([len(x) for x in [df_case.keys(), cbodies_all]])
wg_cases = widgets.SelectMultiple(options = list(df_case.keys()),
                                  value = list(df_case.keys()),
                                  rows = rows_widgets,
                                  disable = False,
                                  layout = Layout(width = '100%')
                                  )

# X-axis
wg_xaxis = widgets.Select(options = x_axis_options,
                          value = x_axis_options[0],
                          rows = rows_widgets,
                          disable = False,
                          layout = Layout(width = '100%')
                          )

# Y-axis
wg_yaxis = widgets.Select(options = sorted(fm.keys(), key = lambda s: (len(s), str.lower)),
                          rows = rows_widgets,
                          value = 'K1_2',
                          disable = False,
                          layout = Layout(width = '100%')
                          )

# Contact bodies
# FOR NOW: Remove not relevant files
remove_cbodies = ['End_plate', 'None']
cbodies_all = [cb for cb in cbodies_all if not any([rb in cb for rb in remove_cbodies])]
cbodies_all.sort()
cbodies_all.sort(key=len)

wg_cbodies = widgets.SelectMultiple(options = list(cbodies_all),
                                    value = ['SUM'],
                                    rows = rows_widgets,
                                    disable = False,
                                    layout = Layout(width = '100%')
                                    )

# Toggle color
toggle_color = widgets.ToggleButtons(options=['Cases', 'Layers'],
                                     description='Color:',
                                     disabled=False,
#                                      layout=Layout(width='40%'),
                                     tooltips=['Different color for each case', 
                                               'Different color for each component'])

# Toggle markers
toggle_markers = widgets.ToggleButtons(options = ['lines', 'lines+markers'],
                                       description='Style',
                                       disabled=False,
                                       tooltips=['Markers on', 'Markers off']
#                                        layout=Layout(width = '60%')
                                      )


# Toggle curv vs timestep
toggle_curv = widgets.ToggleButtons(options=['Curvature', 'Time step'],
                                    description='\t',
                                    value='Time step'
#                                     layout = Layout(width = '60%')
                                    )
toggle_color.style.button_width=toggle_markers.style.button_width=toggle_curv.style.button_width='110px'


# Tabs for naming of plot
tab = widgets.Tab()
plot_names = [cases[name] if name in cases.keys() else name for name in df_case.keys()]
tab.children = [widgets.Text(value=cases[name], placeholder=name) if name in cases.keys() 
                else widgets.Text(value=name, placeholder=name) for name in df_case.keys()]

for i, name in enumerate(df_case.keys()):
        tab.set_title(i, name)
        
# Position and curvature
wg_curv = widgets.BoundedFloatText(value=0.8,
                                   min=0,
                                   max=1,
                                   step=.01,
                                   description='Curvature',
                                   disabled=False
                                  )

wg_timestep = widgets.BoundedFloatText(value=2.0,
                                       min=0.0,
                                       max=2.0,
                                       step=0.01,
                                       description='Timestep',
                                       disabled=False
                                       )

wg_pos = widgets.Dropdown(value=positions_all[-2],
                          options=positions_all,
                          description='Position',
                          disabled=False
                          )

# Make boxes
ui_box1 = widgets.HBox([wg_cases, wg_cbodies, wg_xaxis, wg_yaxis])
ui_box2 = widgets.HBox([toggle_color, toggle_markers, toggle_curv])
ui_box3 = widgets.HBox([wg_pos, wg_curv, wg_timestep])

#### Plot

In [30]:
# Main
run_button = widgets.Button(description='Show below selection',
                            layout=Layout(width = '99%'))

# # Create/clear file for writing results
# res_file= project_folder / 'CS_print_res.csv'
# with open(res_file, 'w') as fo: pass
val = '' # TEST

def run_on_click(a):
    
    '''
    Format of dataframes results are picked from:
     - df with full cross section results = df_case[case][pos]
     - df with excel results = df_results[case]
     - length plot : df_plot consists of one row with results for each position
    
    # Todo: find max inc position
    
    '''
    
    # Create/clear file for writing results
    res_file = project_folder / 'CROSSECTION_post-processing.csv'
    df_write = pd.DataFrame()
    with open(res_file, 'w') as fo: pass
    
    # Read position and curvature from widgets
    curvature = wg_curv.value
    position = wg_pos.value
    timestep = wg_timestep.value
    
    # Initialization
    data = []
    clear_output()
        
    # Find increment with critical curvature
    curv_inc = {}

    for case in wg_cases.value:
#         try:
        df = df_case[case][position]
        curv_inc[case] = (df['K1_2'].abs()-curvature).abs().argsort()[0]
#         except KeyError:
#             pass
    
    # Extract case names from tabs
    case_names = {}
    for c in tab.children:
        case_names[c.placeholder] = c.value
    
    # Plot each case
    for i, case in enumerate(wg_cases.value):
        
        # Create empty dataframe 
        df_plot = pd.DataFrame()
        
        # Get positions for this case
        positions = sorted([int(p) for p in df_case[case].keys()])
        
        #  L E N G T H
        if wg_xaxis.value == 'Length':
            
            # Function of length for given inc - only when curvature not timestep
            inc = curv_inc[case]
            
            for pos in positions:
                # Extract df with only relevant columns (to limit size)
                df_pos = df_filter(df_case[case][str(pos)], keep=list(wg_cbodies.value), remove=remove_cbodies)
                
                # If for specific curvature
                if toggle_curv.value == 'Curvature':
                    df_inc = df_pos.iloc[inc:inc+1].reset_index(drop=True) # extract current increment
                else:
                    # If for specific time step - SHOULD FIND NEAREST TIME STEP IF NO MATCH
                    df_inc = df_pos[np.round(df_pos['time'], 2)==timestep].reset_index(drop=True)
#                     df_inc = df_pos[np.round(df_pos['time'], 3)==0.240].reset_index(drop=True)
                
                # For both                
                df_res = pd.concat([pd.DataFrame({'Position' : [pos]}), df_inc], axis=1) # concat pos and inc
                df_plot = pd.concat([df_plot, df_res], axis=0, ignore_index=True) # add new pos to bottom
#                 print(df_plot)

            df_plot.drop(['inc', 'time'], axis=1, inplace=True)
        
        #  T I M E
        elif any([wg_xaxis.value == 'Time', wg_xaxis.value == 'Curvature']):
            
            # Position to plot is based on user input
            pos = wg_pos.value
            keep_cbodies = [f'{wg_yaxis.value}.{cb}' for cb in wg_cbodies.value]
            df_plot = df_filter(df_case[case][str(pos)], keep=keep_cbodies, remove=remove_cbodies)
            
            # Add stress if desired
            if wg_yaxis.value == 'Stress':
                df_plot['Stress'] = df_stress[case]['Stress.Flextensile1']
#                 print(df_stress[case])
                # THIS IS A QUICK FIX - ALSO CHECK SAME NUMBER OF TIME STEPS?
                # ONLY ADDS FLEXTENSILE1 HARDCODED
        
        #  O T H E R
        else:
            df_plot = df_filter(df_case[case][position], keep='sum', remove=['none', 'frc'])


        # Plot each part
        for k, val in enumerate([l for l in df_plot.columns.values if wg_yaxis.value in l]):
            # Generate name
            try:
                part_name = val.split('.')[1].replace('_', ' ').capitalize()
            except IndexError:
                part_name = val

            # Define color
            line_type = []
            if toggle_color.value == 'Cases':
                line_color, line_symbol = i, k
            else:
                line_color, line_symbol = k, i

            # X-axis options
            
            try:
                case_name = case_names[case]
            except Exception:
                case_name = case
            
            if wg_xaxis.value == 'Curvature':
                plot_name = f'{case_name}<br>({part_name})'
                x_vals = df_plot['K1_2'].abs()
                x_title = 'Curvature [1/m]'
                x_range = [0, 1.0]
                plot_loc = wg_pos.value
                
            elif wg_xaxis.value == 'Time':
                plot_name = f'{part_name}<br>({case_name})'
                x_vals = df_plot['time']
                x_title = 'Time [s]'
                x_range = [0, 2]
                plot_loc = wg_pos.value

            elif wg_xaxis.value == 'Length':
                plot_name = f'{case_name}<br>({part_name})'
                x_vals = df_plot['Position']
                x_title = 'Length [mm]'
                x_range = [0, 1000]
                
                if wg_curv.disabled == False:
                    plot_loc = wg_curv.value
                else:
                    plot_loc = wg_timestep.value
                
            else:
                x_vals = df_results[case_file]['Contact Force']
                x_title = 'Contact force [N]'
                x_range = [0, 300000]          
            
            # Make trace
            trace = go.Scatter(mode=toggle_markers.value,
                               x = x_vals,
#                                y = abs(df_plot[val]),
                               y = df_plot[val],
                               name = plot_name,
                               line = {'color' : colors_4s[line_color]},
                               marker = {'color' : colors_4s[line_color], 
                                         'symbol' : line_symbol, 'size' : 7}
                              )
            data.append(trace)
#             print(len(x_vals))
#             print(list(df_plot[val]))
            
#             # Write to result file
#             try:
#                 csv_file = pd.read_csv(res_file, sep='\t')
#             except Exception:
#                 csv_file = pd.DataFrame()
                
            col_name = f'{case_name.replace("<br>", " - ")} - {plot_loc}'
#             csv_file[f'{col_name} x'] = x_vals
#             csv_file[f'{col_name}'] = df_plot[val]
# #             csv_file.to_csv(res_file, index=False, sep='\t')
            
            # CREATE DF_WRITE INSTEAD
            df_write = pd.concat([df_write, 
                                  pd.DataFrame({f'{col_name}' : x_vals}),
                                  pd.DataFrame({f'{col_name} - {val}' : df_plot[val]})], axis=1)


    # Write to csv file
    df_write.fillna('')
    df_write.to_csv(res_file, index=False, sep='\t')
            
            
    # Y-axis title
    try:
        y_title = fm_units[val.split('.')[0]]
    except (KeyError, UnboundLocalError):
        y_title = val # If here : df_filter keep might be set to 'sum' which removes some variables

    # Change range if desireable
    if wg_yaxis.value == 'EI':
        y_axis_dict = dict(title=y_title, range=[-30e6, 30e6])
    else:
        y_axis_dict = dict(title=y_title)
   
    # Layout of plot
    layout = go.Layout(title=f'12" GP riser - {fm[val.split(".")[0]]}',
                       title_x=0.5,
                       xaxis=dict(title=x_title, range=x_range),
                       yaxis=y_axis_dict)
    
    fig1 = dict(data=data, layout=layout)
    plotly.offline.iplot(go.Figure(fig1))
    display(run_button, ui_box3, ui_box1, tab, ui_box2)

run_button.on_click(run_on_click)

display(run_button, ui_box3, ui_box1, tab, ui_box2)

Button(description='Show below selection', layout=Layout(width='99%'), style=ButtonStyle())

In [31]:
# Read position and curvature from widgets
curvature = wg_curv.value
position = wg_pos.value
timestep = wg_timestep.value

# Initialization
data = []
clear_output()

# Find increment with critical curvature
curv_inc = {}

for case in wg_cases.value:
#         try:
    df = df_case[case][position]
    curv_inc[case] = (df['K1_2'].abs()-curvature).abs().argsort()[0]
#         except KeyError:
#             pass

# Extract case names from tabs
case_names = {}
for c in tab.children:
    case_names[c.placeholder] = c.value

# Plot each case
for i, case in enumerate(wg_cases.value):

    # Create empty dataframe 
    df_plot = pd.DataFrame()

    # Get positions for this case
    positions = sorted([int(p) for p in df_case[case].keys()])

    #  L E N G T H
    if wg_xaxis.value == 'Length':

        # Function of length for given inc - only when curvature not timestep
        inc = curv_inc[case]

        for pos in positions:
            # Extract df with only relevant columns (to limit size)
            df_pos = df_filter(df_case[case][str(pos)], keep=list(wg_cbodies.value), remove=remove_cbodies)

            # If for specific curvature
            if toggle_curv.value == 'Curvature':
                df_inc = df_pos.iloc[inc:inc+1].reset_index(drop=True) # extract current increment
            else:
                # If for specific time step - SHOULD FIND NEAREST TIME STEP IF NO MATCH
                df_inc = df_pos[np.round(df_pos['time'], 2)==timestep].reset_index(drop=True)
#                     df_inc = df_pos[np.round(df_pos['time'], 3)==0.240].reset_index(drop=True)

KeyError: '899'

In [ ]:
df_stress.keys()

In [ ]:
df = pd.DataFrame([(.21, .32), (.01, .67), (.66, .03), (.21, .18)],
                  columns=['dogs', 'cats'])
df2 = pd.DataFrame([(.21, .32), (.01, .67), (.66, .03), (.21, .18), (.34, .17)],
                  columns=['dogs1', 'cats1'])

d = pd.DataFrame()
for data in [df, df2]:
    d = pd.concat([d, pd.DataFrame({data.columns[0] : data.columns[0]})], axis=1)
#     d[data.colummns[0]] = data[data.columns[0]]

In [ ]:
df = pd.DataFrame([(.21, .32), (.01, .67), (.66, .03), (.21, .18)],
                  columns=['dogs', 'cats'])
df2 = pd.DataFrame([(.21, .32), (.01, .67), (.66, .03), (.21, .18), (.34, .17)],
                  columns=['dogs1', 'cats1'])

d = pd.DataFrame()
for data in [df, df2]:
    d = pd.concat([d, pd.DataFrame({data.columns[0] : data[data.columns[0]]})], axis=1)
#     d[data.colummns[0]] = data[data.columns[0]]

In [ ]:
d = [1, 2, 3]

In [ ]:
df.columns[0]

#### Time plot V2

In [ ]:
# To be checked

In [ ]:
# Settings
fm = {'Fx' : 'Force X', 'Fy' : 'Force Y', 'Fz' : 'Force Z',
      'Mx' : 'Moment X', 'My' : 'Moment Y', 'Mz' : 'Moment Z',
      'Av Pl Strain' : 'Average plastic strain',
      'Max Pl Strain' : 'Max plastic strain'}

colors_4s = ['#00a0b0', '#f3776f', '#012b5d', '#009ddc', '#00a88f', '#faa634', '#c0c1ba', '#f8ed9b',
             '#00a0b0', '#f3776f', '#012b5d', '#009ddc', '#00a88f', '#faa634', '#c0c1ba', '#f8ed9b']
line_styles = ['lines', 'lines+markers', 'markers', 'lines', 'lines+markers', 'markers', 'lines', 'lines+markers', 'markers']

In [ ]:
# Input
folder = r'C:\Projects\1324_NOV_end-fitting_FE_analyses\Analysis\Volve_new'
model_name = 'Volve_liner_holder_new_v2_t2_job' #'Volve_liner_holder_new_v2_t2_6teeth'
case_description = {'case01' : '6 teeth engaging', 'case02' : 'Carcass as eq. cylinder', 
                    'new' : 'Base case'}

element_sets = {'fbr' : 'Front brick right', 'fbl' : 'Front brick left',
                'rbr' : 'Rear brick right',  'rbl' : 'Rear brick left'}

#find csv files
csv_files = find_files(folder, model_name, 'elset_sum', '.csv')
df_dict = {} # format is {node : dataframe}
csv_names = []

#read csv files into data frame
for csv_file in csv_files:
    csv_name = [f.replace('elset', '') for f in  csv_file.split('_') if 'elset' in f][0]
    csv_names.append(csv_name)
    df = pd.read_csv(os.path.join(folder, csv_file))
    
    # Find element data: number, names and order
    df_columns = df.columns.values
    elsets = [e for e in df_columns if 'Element Set' in e]
    no_elsets = len(elsets)
    name_elsets = [df[el][1] for el in elsets]
    
    #change title with elset names
    
    # first find columns that occur more than once
    df_columns_count = collections.Counter([i.split('.')[0] for i in df_columns])
    df_columns_multiple = [c for c, i in df_columns_count.iteritems() if i > 1]
    
    #then for each column, if name occur 
    
    df_columns_new = []
    for col in df_columns:
        if col in df_columns_multiple: # only true for first element set
            df_columns_new.append('_'.join([col, name_elsets[0]]))
        elif len(col.split('.')) == 2: # only true for remaining element sets
            current_elset = int(col.split('.')[1])
            df_columns_new.append('_'.join([col, name_elsets[current_elset]]))
        else:
            df_columns_new.append(col)
    
    df.columns = df_columns_new
    
    df_dict[csv_name] = df
    
# Widgets
cases = widgets.SelectMultiple(options = csv_names,
                               value = csv_names,
                               disable = False,
                               layout = Layout(width = '100%')
                               )

plot_fm = widgets.SelectMultiple(options = fm.keys(),
                                 value = fm.keys(),
                                 disable = False,
                                 layout = Layout(width = '100%')
                                 )

run_button = widgets.Button(description='Show below selection', 
                            layout=Layout(width = '97%')
                           )

ui = widgets.HBox([cases, plot_fm])
display(run_button, ui)

df_force = pd.read_csv(os.path.join(folder, 'v2_t2_reaction_forcy_y.csv'))

#plot
def run_on_click(a):
    data = []
    clear_output()
    
    for i, case in enumerate(cases.value): # plot each case
        for j, plotme in enumerate(plot_fm.value): # plot each results (only one?)
            for k, val in enumerate([l for l in df_dict[case].columns.values if plotme in l]): # plot each element set
                elset_name = [val.split('_')[-1].replace(ro, rn) for ro, rn in element_sets.iteritems() if ro in val.split('_')[-1]][0]
                trace = go.Scatter(mode=line_styles[i],
                                   x = df_force['0'],
                                   y = df_dict[case][val],
                                   name = '{} <br>{}'.format(fm[plotme], elset_name),
                                   line = {'color' : colors_4s[k]})
                data.append(trace)

    layout = go.Layout(xaxis=dict(title='Pull force [N]'),
                       yaxis=dict(title='Force [N]'))#' / Moment [Nmm] / Strain [-]'))
    
    plotly.offline.iplot(go.Figure(data=data, layout=layout))
    display(run_button, ui)

run_button.on_click(run_on_click)

#### Another plot

In [ ]:
# Excel file with results to be read
excel_file = '1535_csv_new.xlsx'
extract_results = ['Time', 'Contact Force', 'Stress', 'Moment']
excel_file_dir = project_folder / excel_file

# Extract results
df_results = read_excel_results(excel_file_dir, extract_results, cases)

# OPTIONAL - extract local results
try:
    if isinstance(cases_local, dict):
        df_local = read_excel_results(excel_file_dir, extract_results, cases_local)
except NameError:
    pass

# Plot each case in df_results['Contact Force']
data = []
for i, case in enumerate(df_results.keys()):
    trace = go.Scatter(mode=line_styles[0],
                       x = df_results[case]['Time'],
                       y = df_results[case]['Contact Force'],
                       name = case,
                       line = {'color' : colors_4s[i]}
                      )
    data.append(trace)

# Define layout of plot
layout = go.Layout(title='Contact force comparison',
                   xaxis=dict(title='Time [s]', range=[1.2, 1.8]),
                   yaxis=dict(title='Contact force [N]', range=[0, 300000]))

# plotly.offline.iplot(go.Figure(data=data, layout=layout))

## Axisymmetric

### Time plot

In [ ]:
df = pd.DataFrame([[0, 1, 2, 3], [3, 4, 5, 6]], columns = ['t', 'a', 'b', 'c'])
df[df['t']==3]

In [ ]:
# Layers to remove from plot
rem_layers = ['SUM', 'Carcass']

# Input
diff_pres = 500 # differential pressure
skip_incs = 10 # number of increments in pre load case

In [ ]:
# Sort of list with name of runs
runs = find_files(folder_path, 'csv') # find files containing .csv files in subfolder
runs = sorted(set([r.parts[0] for r in runs])) # sort out run names

# Make buttons with names of list
run_list = wg.ToggleButtons(
    options=runs,
    disabled=False,
    button_style='',
    layout=Layout(width = '75%')
)

# Make run button to run post-processing
run_button = wg.Button(description='Run post-processing', 
                            layout=Layout(width = '75%'))
run_button.style.button_color='#23DFAC'

# Display buttons
ui = wg.VBox([run_button, run_list])
display(ui)

# Define function that should run on click
def run_on_click(a):    
    # input
    case_path = folder_path / run_list.value
    
    # make figure
    clear_output()
    figure = {
        'data': [],
        'layout': {},
        'frames': []
    }
    
    # read csv-file and axial forces from file
    force = read_results(case_path, 'Fx')

    # definitions
    time_steps = list(next(iter(force.values())).count())[0] - skip_incs # skip inc 0 and X increments
    pressures = list(np.round([x / (time_steps-1) * diff_pres for x in range(0, time_steps)], 0))
    pressures= [int(l) for l in pressures] # Can not be float decimal number
    layers = trim_list(list(force.keys()), rem_layers)
    
    # ranges
    x_range, y_range = find_ranges(force)
    #y_range[0] = 0
    
    # define sliders
    sliders_layout, sliders_dict = make_sliders(pressures)

    # fill in most of layout
    figure['layout'] = dict(title = run_list.value,
                            xaxis = {'range': x_range, 
                                     'title': 'Position [mm]'},
                            yaxis = {'range': y_range, 'title': 
                                     'Axial Force [N]'},
                            height = 650,
                            hovermode = 'closest',
                            sliders = sliders_layout,
                            updatemenus = add_play_pause()
                           )
  
    # make data
    for layer in layers:
        # skipping first 10 inc (before differential pressure is applied)
        data_dict = dict(x=list(force[layer].columns),
                         y=list(force[layer].loc[skip_incs]),
                         name=layer
                        )
        data_dict
        figure['data'].append(data_dict)

    # make frames
    for i, pressure in enumerate(pressures, skip_incs):
        frame = dict(data=[],
                     name=pressure)

        for layer in layers:
            data_dict = dict(x=list(force[layer].columns),
                             y=list(force[layer].loc[i]),
                             name=layer
                            )
            frame['data'].append(data_dict)

        figure['frames'].append(frame)

        slider_step = {'args': [
            [pressure],
            {'frame': {'duration': 300, 'redraw': False},
             'mode': 'immediate',
           'transition': {'duration': 300}}
         ],
         'label': str(pressure) + ' bar',
         'method': 'animate'}
        
        sliders_dict['steps'].append(slider_step)

    figure['layout']['sliders'] = [sliders_dict]
    
    # Display buttons and figure
    plotly.offline.iplot(figure)
    display(ui)
    
run_button.on_click(run_on_click)

In [ ]:
sorted(run_dir.rglob(f'*{arg}*')

## Old

### From 1324 project

In [ ]:
import os
import pandas as pd
import plotly
import plotly.graph_objs as go
import collections
plotly.offline.init_notebook_mode(connected=True)

# increase width of screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:76% !important; }</style>"))

# 4s colors
colors_4s = ['#00a0b0', '#f3776f', '#012b5d', '#009ddc', '#00a88f', '#faa634', '#c0c1ba', '#f8ed9b',
             '#00a0b0', '#f3776f', '#012b5d', '#009ddc', '#00a88f', '#faa634', '#c0c1ba', '#f8ed9b']


def find_files(run_dir, *args):
    '''
    Function that finds all files containing arguments in directory
    
    :param run_dir: Directory containing files
    :param args: Arguments which run_dir must contain
    
    :return files: Files containing arguments
    '''
    files = []
    
    for (dir_path, dir_names, file_names) in os.walk(run_dir):
        for file in file_names:
            file_path = os.path.relpath(os.path.join(dir_path, file), run_dir)
            
            # Filter to paths containing arg
            if all(arg.lower() in file_path.lower() for arg in args):
                files.append(file_path)
    
    return files


def filter_files(file_list, filter_list):
    return [l for l in file_list if not any([s in l for s in filter_list])]


def get_max_min(df, get):
    
    if get.lower() == 'max':
        val_loc = list(df.max()).index(max(df.max()))
    elif get.lower() == 'min':
        val_loc = list(df.min()).index(min(df.min()))
    else:
        return False
    
    val_name = df.columns[val_loc].split('.')[0]
    return_df = df.iloc[:, val_loc]
    
    return val_name, return_df
    
    
def remove_after(var, symbol):
    if symbol in var:
        var_split = var.split(symbol)
        var = symbol.join(var_split[:-1])
    return var


def rename_cols(columns):
    rename_cols = {}
    for col in columns:
        col_val = col.replace(title, '').replace('.', '')
        try:
            col_val = int(col_val) + 1
        except ValueError:
            if col_val == '':
                col_val = 1
            else:
                pass
        rename_cols[col] = col_val
    return rename_cols

# input
folder = r'C:\Projects\1324_NOV_end-fitting_FE_analyses\Previous_project\3D_FE_capacity_Aasgard\temp_pos12'
model_name = 'asgard_3d_liner_holder_pos12_new' # extract results from ALL models with this name
read_names = {'asgard_3d' : 'Aasgard 12deg'} # change name to readable name (order dependent)
contact_bodies = {}
filter_strings = ['t1', 't3_job1'] # names that files should not contain

# initialize variables to be used
data = []
cs_results = {}
post_results = {}
results = {}

# Sort of list with name of runs
runs = find_files(folder, 'wire', '.csv') # find files containing .csv files in subfolder
runs = sorted(set([r.split('\\')[0] for r in runs])) # sort out run names

files = find_files(folder, model_name, 'wire_cb')
files = filter_files(files, filter_strings)
models = list(set(['_'.join(f.split('_')[0:-4]) for f in files]))
models = filter_files(models, filter_strings)

# Looping through each FE model results files
for model in models:
    # initialize variable and find .csv files
    post_res = pd.DataFrame()
    files = [f for f in files if model in f]
    
    # Looping through each contact body
    for file in files:
        dict_res = {}
        # find number of contact body and corresponding name 
        cb_number = file[file.find('wire_cb')+7:file.find('wire_cb')+9].replace('.','')
        cb_name = contact_bodies[cb_number]
        
        # find columns containing stress and unique names
        df = pd.read_csv(os.path.join(folder, file))
        df_stress = df.loc[:, df.columns.str.contains('Stress')]
        # stresses = list(set([l.split('.')[0] for l in df_stress.columns.values])) # if to find max in each corner
        stresses = ['Stress'] # if to find max of all corners
        
        post_res['Time'] = df['time']
        post_res['Curvature'] = df['Curvature K2']
        
        for stress in stresses:
            df_post = df_stress.loc[:, df_stress.columns.str.contains(stress)]
    
            for var in ['Max', 'Min']:
                var_name, df_post = get_max_min(df, var)
                df_name = ' '.join([cb_name, '<br>', var_name, '-', var])
                post_res[df_name] = df_post

        # THIS READS IN A BIT MORE TIDY WAY: results[model][contact_body][variable] = df with values
        column_titles = [remove_after(name, '.') for name in df.columns.values]
        titles = collections.Counter(column_titles)
        for title, count in titles.items():
            if 'Curvature' in title:
                df_temp = df.loc[:, df.columns == title]
            else:
                df_temp = df.loc[:, df.columns.str.contains(title)]
            dict_res[title] = df_temp.rename(columns=rename_cols(df_temp.columns))
        
        results[' '.join([model, cb_name])] = dict_res
    post_results[model] = post_res

# Plotting
for case, df in post_results.items():
    for res in df.columns.values:
        if any(w in res.lower() for w in ['max', 'min']):
            # if multiple models - incldue name and change name to readable name if possible
            if len(post_results.keys()) > 0:
                for key, val in read_names.items():
                    if key in case:
                        read_name = val
                        break
                    else:
                        read_name = case
                name = '<br>'.join([read_name, res])
            else:
                name = res
            
            # make plot
            trace = go.Scatter(
                mode='lines',
                x=[abs(m) for m in df['Curvature']],
                y=df[res],
                name=name,
                line = {'color' : colors_4s}
            )
            data.append(trace)

layout = go.Layout(title = 'Stress in tensile armour',
                   xaxis=dict(title='Curvature [1/m]'),
                   yaxis=dict(title='Stress [MPa]'),
                   height=650)


plotly.offline.iplot(go.Figure(data=data, layout=layout))

### Animation related functions

In [ ]:
def make_sliders(pressures):
    '''
    Function that defines the sliders
    
    :list pressures: List with pressure for each increment
    
    :return sliders_layout: Dictionary with layout for slider
    :return sliders_dict: Dictionary with slider information
    '''
    incs = list(range(0, len(pressures)))
    sliders_layout = {
        'args': [
            'transition', {
                'duration': 400,
                'easing': 'cubic-in-out'
            }
        ],
        'initialValue': '0.0',
        'plotlycommand': 'animate',
        'values': pressures,
        'visible': True
    }
    
    sliders_dict = {
        'active': 0,
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Differential pressure: ',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 500, 'easing': 'cubic-in-out'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': []
    }
    
    return sliders_layout, sliders_dict


def add_play_pause():
    '''
    Function that adds play and pause buttons to plot
    
    :return play_pause_buttons: list with information about play and pause buttons.    
    '''
    play_pause_buttons = [
        {
            'buttons': [
                {
                    'args': [None, {'frame': {'duration': 500, 'redraw': False},
                             'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                    'label': 'Play',
                    'method': 'animate'
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                    'transition': {'duration': 0}}],
                    'label': 'Pause',
                    'method': 'animate'
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }
    ]
    
    return play_pause_buttons